<img src="https://fsdl.me/logo-720-dark-horizontal">

This notebook accompanies [this YouTube video](https://www.youtube.com/watch?v=zaYTXQFR0_s)
walking through what LangChain is and interviewing the creator, Harrison Chase.

## Auth

In [33]:
%pip install -qqq langchain openai
%pip install -qqq beautifulsoup4
%pip install -qqq unstructured
%pip install -qqq tiktoken
%pip install -qqq faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import os
from dotenv import load_dotenv
load_dotenv()

#set up for tracing
os.environ["LANGCHAIN_HANDLER"] = "langchain"

In [41]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(r'C:\Users\jonma\github_repos\freemocap_organization\documentation\docs', glob="**/*.md")
documents = loader.load()
print(documents)

[Document(page_content="About Us\n\nThe Free Motion Capture Project (FreeMoCap) aims to provide research-grade markerless motion capture software to everyone for free.\n\nWe're building a user-friendly framework that connects an array of bleeding edge open-source tools from the computer vision and machine learning communities to accurately record full-body 3D movement of humans, animals, robots, and other objects.\n\nWe want to make the newly emerging mind-boggling, future-shaping technologies that drive FreeMoCap's core functionality accessible to communities of people who stand to benefit from them.\n\nWe follow a “Universal Design” development philosophy, with the goal of creating a system that serves the needs of a professional research scientist while remaining intuitive to a 13-year-old with no technical training and no outside assistance.\n\nA high-quality, minimal-cost motion capture system would be a transformative tool for a wide range of communities - including 3d animators,

In [44]:


from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()



In [45]:


from langchain.vectorstores import FAISS

docsearch = FAISS.from_documents(documents, embeddings)



### Ask questions and get answers

In [47]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain


from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0)



chain = load_qa_with_sources_chain(llm, chain_type="stuff")

In [48]:
query = "What is FreeMoCap? Does it have realtime capacity? Who made it? What is a charuco board?"
# query = "What is LangChainHub?"
# query = "Does LangChain integrate with OpenAI? If so, how?"

docs = docsearch.similarity_search(query)
result = chain({"input_documents": docs, "question": query})

text = "\n".join(textwrap.wrap(result["output_text"]))
text = "\n\nSOURCES:\n".join(map(lambda s: s.strip(), text.split("SOURCES:")))

print(text)

FreeMoCap is a project that aims to provide research-grade markerless
motion capture software for free. It connects various open-source
tools from the computer vision and machine learning communities to
accurately record full-body 3D movement of humans, animals, robots,
and other objects. The current documentation does not mention real-
time capacity. A charuco board is used for calibrating the capture
volume in the motion capture process. The creators of FreeMoCap are
not explicitly mentioned in the provided content.

SOURCES:
C:\Users\jo
nma\github_repos\freemocap_organization\documentation\docs\about_us.md
, C:\Users\jonma\github_repos\freemocap_organization\documentation\doc
s\terminology\terminology.md


In [ ]:
print(*textwrap.wrap(result["input_documents"][0].page_content), sep="\n")